# Deep mutational scanning of ZIKV E protein
Deep mutational scanning of ZIKV E from the MR766 strain.
Experiments performed by the [Matt Evans lab](http://labs.icahn.mssm.edu/evanslab/).
Sequencing and computational analyses performed by the [Bloom lab](https://research.fhcrc.org/bloom/en.html).

## Set up for analysis
Import Python packages and modules:

In [1]:
import glob
import os
import subprocess

import pandas
from IPython.display import display, HTML

import dms_tools2
from dms_tools2.ipython_utils import showPDF
print(f"Using dms_tools2 {dms_tools2.__version__}")

Using dms_tools2 2.3.0


Specify information about running analysis:

In [2]:
use_existing = 'yes' # use existing output

ncpus = 4 # max CPUs to use

# directories
resultsdir = './results/'
os.makedirs(resultsdir, exist_ok=True)

Input data found in the [./data/](data) directory:

In [3]:
refseqfile = './data/E.fasta' # sequence of wildtype gene
samplelist = './data/samplelist.csv' # samples sequenced
alignspecsfile = './data/subamplicon_alignspecs.txt'

## Process deep sequencing data
We process the data from the [barcoded subamplicon deep sequencing](https://jbloomlab.github.io/dms_tools2/bcsubamp.html) to count the frequency of each codon in each sample.

First, we read in the samples:

In [4]:
samples = (pandas.read_csv(samplelist)
           .assign(name=lambda x: x.library + '-' + x.selection)
           )

display(HTML(samples.to_html(index=False)))

library,selection,antibody,percent_infectivity,R1,name
Lib1,plasmid,none,NaN,/fh/fast/bloom_j/SR/ngs/illumina/bloom_lab/180...,Lib1-plasmid
Lib2,plasmid,none,NaN,/fh/fast/bloom_j/SR/ngs/illumina/bloom_lab/180...,Lib2-plasmid
Lib3,plasmid,none,NaN,/fh/fast/bloom_j/SR/ngs/illumina/bloom_lab/180...,Lib3-plasmid
wildtype,plasmid,none,NaN,/fh/fast/bloom_j/SR/ngs/illumina/bloom_lab/180...,wildtype-plasmid
Lib1,virus,none,NaN,/fh/fast/bloom_j/SR/ngs/illumina/bloom_lab/180...,Lib1-virus
Lib2,virus,none,NaN,/fh/fast/bloom_j/SR/ngs/illumina/bloom_lab/180...,Lib2-virus
Lib3,virus,none,NaN,/fh/fast/bloom_j/SR/ngs/illumina/bloom_lab/180...,Lib3-virus
wildtype,virus,none,NaN,/fh/fast/bloom_j/SR/ngs/illumina/bloom_lab/180...,wildtype-virus
Lib1,no-antibody-A,no-antibody,71.06000,/fh/fast/bloom_j/SR/ngs/illumina/bloom_lab/180...,Lib1-no-antibody-A
Lib1,no-antibody-B,no-antibody,131.89000,/fh/fast/bloom_j/SR/ngs/illumina/bloom_lab/180...,Lib1-no-antibody-B


Now we read in the alignment specs for the [barcoded subamplicon sequencing](https://jbloomlab.github.io/dms_tools2/bcsubamp.html):

In [5]:
with open(alignspecsfile) as f:
    alignspecs = f.read().strip()
print(alignspecs)

1,303,33,38 304,609,38,40 610,903,41,36 904,1200,41,37 1201,1512,36,35


Now we use the [dms2_batch_bcsubamp](https://jbloomlab.github.io/dms_tools2/dms2_batch_bcsubamp.html) program to process the deep sequencing data to obtain codon counts:

In [6]:
countsdir = os.path.join(resultsdir, 'codoncounts')
os.makedirs(countsdir, exist_ok=True)

bcsubamp_batchfile = os.path.join(countsdir, 'batch.csv')
samples[['name', 'R1']].to_csv(bcsubamp_batchfile, index=False)

subprocess.check_call([
    'dms2_batch_bcsubamp',
    '--batchfile', bcsubamp_batchfile,
    '--refseq', refseqfile,
    '--alignspecs', alignspecs,
    '--outdir', countsdir,
    '--summaryprefix', 'summary',
    '--R1trim', '200',
    '--R2trim', '200',
    '--ncpus', str(ncpus),
    '--use_existing', use_existing
    ])

0